In [91]:
import os
import fastf1
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [98]:
def extract_data(track):
    session = fastf1.get_session(2024, track, 'R')
    session.load()

    lap = session.laps.pick_fastest()
    pos = lap.get_pos_data()

    circuit_info = session.get_circuit_info()

    return lap, pos, circuit_info

In [99]:
circuits = [
    'Imola', 
    'Monza', 
    'Mexico City', 
    'Bahrain', 
    'Baku', 
    'Barcelona', 
    'Monaco', 
    'Circuit de Spa-Francorchamps', 
    'Montreal', 
    'Austin', 
    'Zandvoort', 
    'Hungary', 
    'Interlagos', 
    'Jeddah', 
    'Las Vegas', 
    'Lusail', 
    'Singapore', 
    'Melbourne', 
    'Miami', 
    'Austria', 
    'Shanghai', 
    'Silverstone', 
    'Suzuka', 
    'Abu Dhabi'
]


In [ ]:
# Create a directory to store the data
data_dir = "all_data"
os.makedirs(data_dir, exist_ok=True)

# Iterate through each circuit and extract data
for track in circuits:
    try:
        print(f"Extracting data for {track}...")
        
        # Extract the data
        lap, pos, circuit_info = extract_data(track)
        
        # Save the data to a file
        file_path = os.path.join(data_dir, f"{track.replace(' ', '_').replace('/', '_')}_data.pkl")
        
        with open(file_path, 'wb') as f:
            pickle.dump({'lap': lap, 'pos': pos, 'circuit_info': circuit_info}, f)
        
        print(f"Data for {track} saved successfully.")

    except Exception as e:
        print(f"Error extracting data for {track}: {e}")

In [101]:
# Define the rotation function
def rotate(xy, *, angle):
    rot_mat = np.array([[np.cos(angle), np.sin(angle)],
                        [-np.sin(angle), np.cos(angle)]])
    return np.matmul(xy, rot_mat)

In [172]:
# Set the directory where the pickle files are stored
data_dir = "all_data"

# Prepare a list to hold the data
rotated_data = []

# Iterate through each pickle file in the data directory
for idx, file_name in enumerate(os.listdir(data_dir)):
    if file_name.endswith('.pkl'):
        # Load the data from the pickle file
        file_path = os.path.join(data_dir, file_name)
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        
        # Extract the track name (removing file extension)
        track_name = file_name.replace('_data.pkl', '').replace('_', ' ')
        
        # Extract the position data (X, Y)
        track = data['pos'].loc[:, ('X', 'Y')].to_numpy()
        
        # Extract the rotation angle
        track_angle = data['circuit_info'].rotation / 180 * np.pi
        
        # Rotate the track data
        rotated_track = rotate(track, angle=track_angle)
        
        # Create a point_id for each point in the track
        for i in range(rotated_track.shape[0]):
            rotated_data.append([track_name, i + 1, rotated_track[i, 0], rotated_track[i, 1]])

# Convert the list of rotated data to a DataFrame
df = pd.DataFrame(rotated_data, columns=['track_name', 'point_id', 'rotated_x', 'rotated_y'])

In [173]:
schedule = fastf1.get_event_schedule(2025)  # Get the 2024 season schedule

race_sched_df = schedule[schedule['Session5'] == 'Race'][['Location', 'EventDate']]
race_sched_df = race_sched_df.reset_index(drop=True)

race_sched_df.sort_values(by=['EventDate'], ascending=True, inplace=True)

# Number of rows and columns for the small multiple (6x4 grid)
num_rows = 4
num_cols = 6

# Add row_id and column_id based on the index
race_sched_df['row_id'] = (race_sched_df.index // num_cols) + 1  # Row ID (starts from 1)
race_sched_df['column_id'] = (race_sched_df.index % num_cols) + 1  # Column ID (starts from 1)
print(race_sched_df)


             Location  EventDate  row_id  column_id
0           Melbourne 2025-03-16       1          1
1            Shanghai 2025-03-23       1          2
2              Suzuka 2025-04-06       1          3
3              Sakhir 2025-04-13       1          4
4              Jeddah 2025-04-20       1          5
5               Miami 2025-05-04       1          6
6               Imola 2025-05-18       2          1
7              Monaco 2025-05-25       2          2
8           Barcelona 2025-06-01       2          3
9            Montréal 2025-06-15       2          4
10          Spielberg 2025-06-29       2          5
11        Silverstone 2025-07-06       2          6
12  Spa-Francorchamps 2025-07-27       3          1
13           Budapest 2025-08-03       3          2
14          Zandvoort 2025-08-31       3          3
15              Monza 2025-09-07       3          4
16               Baku 2025-09-21       3          5
17         Marina Bay 2025-10-05       3          6
18          

In [174]:
circuit_mapping = {
    'Imola': 'Imola',
    'Monza': 'Monza',
    'Mexico City': 'Mexico City',
    'Bahrain': 'Sakhir',
    'Baku': 'Baku',
    'Barcelona': 'Barcelona',
    'Monaco': 'Monaco',
    'Circuit de Spa-Francorchamps': 'Spa-Francorchamps',
    'Montreal': 'Montréal',
    'Austin': 'Austin',
    'Zandvoort': 'Zandvoort',
    'Hungary': 'Budapest',
    'Interlagos': 'São Paulo',
    'Jeddah': 'Jeddah',
    'Las Vegas': 'Las Vegas',
    'Lusail': 'Lusail',
    'Singapore': 'Marina Bay',
    'Melbourne': 'Melbourne',
    'Miami': 'Miami',
    'Austria': 'Spielberg',
    'Shanghai': 'Shanghai',
    'Silverstone': 'Silverstone',
    'Suzuka': 'Suzuka',
    'Abu Dhabi': 'Yas Island'
}

In [175]:
df['alias'] = df['track_name'].map(circuit_mapping)

In [176]:
df.head()

,track_name,point_id,rotated_x,rotated_y,alias
0,Abu Dhabi,1,2368.041982,-939.060261,Yas Island
1,Abu Dhabi,2,2400.640375,-1049.070918,Yas Island
2,Abu Dhabi,3,2478.749016,-1313.525149,Yas Island
3,Abu Dhabi,4,2576.891491,-1645.526737,Yas Island
4,Abu Dhabi,5,2602.194257,-1731.438145,Yas Island


In [177]:
# Perform the left join on 'Circuit'
df = df.merge(race_sched_df, left_on='alias',right_on='Location', how='left')

In [178]:
df.drop(columns=['Location'], inplace=True)

In [179]:
df.head()

,track_name,point_id,rotated_x,rotated_y,alias,EventDate,row_id,column_id
0,Abu Dhabi,1,2368.041982,-939.060261,Yas Island,2025-12-07,4,6
1,Abu Dhabi,2,2400.640375,-1049.070918,Yas Island,2025-12-07,4,6
2,Abu Dhabi,3,2478.749016,-1313.525149,Yas Island,2025-12-07,4,6
3,Abu Dhabi,4,2576.891491,-1645.526737,Yas Island,2025-12-07,4,6
4,Abu Dhabi,5,2602.194257,-1731.438145,Yas Island,2025-12-07,4,6


In [180]:
df.sort_values(by=['EventDate','point_id'], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [181]:
df.head()

,track_name,point_id,rotated_x,rotated_y,alias,EventDate,row_id,column_id
0,Melbourne,1,-169.867874,-1790.530900,Melbourne,2025-03-16,1,1
1,Melbourne,2,-285.889768,-1792.749018,Melbourne,2025-03-16,1,1
2,Melbourne,3,-619.741425,-1795.408189,Melbourne,2025-03-16,1,1
3,Melbourne,4,-855.258505,-1796.247725,Melbourne,2025-03-16,1,1
4,Melbourne,5,-1057.558970,-1798.374273,Melbourne,2025-03-16,1,1


In [182]:
# Save the DataFrame to a CSV file
output_file = 'all_rotated_track_data.csv'
df.to_csv(output_file, index=False)

print(f"Rotated track data with row_id and column_id saved to {output_file}")

Rotated track data with row_id and column_id saved to all_rotated_track_data.csv
